In [7]:
import os
import sys

sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname('__file__'), '..')))
from src.encoding.tools import create_encoding_pipeline
from src.encoding.encoders import *
from src.datasets.base_tabular_dataset import BaseTabularDataset
import src.features as ft

import pandas as pd

from sklearn import set_config

import logging

ModuleNotFoundError: No module named 'sklearn'

In [2]:
import numpy as np


In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO, encoding="utf-8",
                    format="%(name)s %(asctime)s: %(levelname)s: %(message)s", handlers=[logging.StreamHandler()])
root_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
config = ft.Config({'max_nan': 0, "departement": "21", "root_dir": root_dir, "start": '01-01-2019',
                    "stop": '31-12-2023', "logger": logger, "step_unit": 'days', "step_value": 1,
                    "shift": 0, "rolling_window": 0, "etablissement": "CHU Dijon", 'region':'BOURGOGNE'})

ars_features_class = [ft.AirQualityFeatures, ft.HopitalFeatures(config=config, include_emmergency_arrivals=True, include_nb_hospit=True), ft.EpidemiologicalFeatures, ft.FireFightersFeatures(config=config, include_calls=False),
                      ft.GoogleTrendFeatures, ft.MeteorologicalFeatures, ft.SociologicalFeatures,
                      ft.SportsCompetitionFeatures, ft.TrafficFeatures]
baseTabularDataset = BaseTabularDataset(target_colomns='Total_CHU Dijon',
                                    config=config, features_class=ars_features_class)

baseTabularDataset.fetch_data()

In [ ]:
dataset = baseTabularDataset.get_dataset(from_date=dt.datetime.strptime('01-01-2019', '%d-%m-%Y'), to_date=dt.datetime.strptime('31-12-2023', '%d-%m-%Y'), shift=7, rolling_window=[7, 14], create_X_y=True, split_config = {'test_size': 0.2, 'val_size': 0.2, 'shuffle': False})


In [4]:
encoders_dict = {
    'number': {
        'as_number': {
            'imputers': [imputers.SimpleImputer(strategy='mean')],
            'encoders': [
                ne.StandardScaler(),

            ]
        }
    },
    'category': {
        'as_category': {
            'imputers': [imputers.SimpleImputer(strategy='most_frequent')],
            'encoders': [
                ne.MultiTargetEncoder(drop_invariant=True, return_df=True),
            ]
        }
    },
    'datetime': {
        'as_number': {
            'imputers': [de.DateFeatureExtractor()],
            'encoders': [
                ne.CyclicalFeatures(drop_original=True)
            ]
        },
        'as_category': {
            'imputers': [de.DateFeatureExtractor(dtype='category')],
            'encoders': [
                ne.MultiTargetEncoder(drop_invariant=True, return_df=True),
            ]
        }
    }
}

In [5]:
processor = create_encoding_pipeline(encoders_dict)

Creating encoding pipeline


In [6]:
processor

FeatureUnion(transformer_list=[('columntransformer-1',
                                ColumnTransformer(transformers=[('pipeline',
                                                                 Pipeline(steps=[('featureunion-1',
                                                                                  FeatureUnion(transformer_list=[('simpleimputer',
                                                                                                                  SimpleImputer())])),
                                                                                 ('featureunion-2',
                                                                                  FeatureUnion(transformer_list=[('standardscaler',
                                                                                                                  StandardScaler())]))]),
                                                                 <sklearn.compose._column_transformer.make_column_selector obj...
                                ColumnTransformer(transformers=[('pipeline',
                                                                 Pipeline(steps=[('featureunion-1',
                                                                                  FeatureUnion(transformer_list=[('datefeatureextractor',
                                                                                                                  DateFeatureExtractor(dtype='category'))])),
                                                                                 ('featureunion-2',
                                                                                  FeatureUnion(transformer_list=[('multitargetencoder',
                                                                                                                  MultiTargetEncoder(drop_invariant=True))]))]),
                                                                 <sklearn.compose._column_transformer.make_column_selector object at 0x70f3da70b2e0>)]))],
             verbose=True)

In [7]:
for target in y.columns:
    processor.fit(data, y)
    processor

[FeatureUnion]  (step 1 of 4) Processing columntransformer-1, total=   0.0s
[FeatureUnion]  (step 2 of 4) Processing columntransformer-2, total=   0.0s
[FeatureUnion]  (step 3 of 4) Processing columntransformer-3, total=   0.0s
[FeatureUnion]  (step 4 of 4) Processing columntransformer-4, total=   0.1s
[FeatureUnion]  (step 1 of 4) Processing columntransformer-1, total=   0.0s
[FeatureUnion]  (step 2 of 4) Processing columntransformer-2, total=   0.0s
[FeatureUnion]  (step 3 of 4) Processing columntransformer-3, total=   0.0s
[FeatureUnion]  (step 4 of 4) Processing columntransformer-4, total=   0.1s


In [8]:
encoded = processor.transform(data)
print(encoded)

   pipeline__standardscaler__simpleimputer__num_feature1  \
0                                          -1.341641       
1                                          -0.447214       
2                                           0.447214       
3                                           1.341641       

   pipeline__standardscaler__simpleimputer__num_feature2  \
0                                          -1.341641       
1                                          -0.447214       
2                                           0.447214       
3                                           1.341641       

   pipeline__simpleimputer__cat_feature1_target2  \
0                                       0.475633   
1                                       0.453068   
2                                       0.475633   
3                                       0.453068   

   pipeline__simpleimputer__cat_feature2_target2  \
0                                       0.475633   
1                                

In [9]:
encoded = processor.transform(test_data)
print(encoded)

   pipeline__standardscaler__simpleimputer__num_feature1  \
0                                           2.236068       

   pipeline__standardscaler__simpleimputer__num_feature2  \
0                                           2.236068       

   pipeline__simpleimputer__cat_feature1_target2  \
0                                       0.475633   

   pipeline__simpleimputer__cat_feature2_target2  \
0                                       0.475633   

   pipeline__cyclicalfeatures__date_feature_month_sin  \
0                                      -2.449294e-16    

   pipeline__cyclicalfeatures__date_feature_month_cos  \
0                                                1.0    

   pipeline__cyclicalfeatures__date_feature_day_sin  \
0                                     -2.449294e-16   

   pipeline__cyclicalfeatures__date_feature_day_cos  \
0                                               1.0   

   pipeline__cyclicalfeatures__date_feature_dayofweek_sin  \
0                                  